In [1]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


In [2]:
df = pd.read_csv('flo_dataset_augmented.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,...,x6_zinc stearate,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,0,0,0,0.154575,-0.011188,-0.247025,-0.673379,-0.370637,-0.096002,...,1,0,1,0,0,0,0,1.750000,480.0,539.0
1,1,1,1,1,0.195883,0.280681,-0.439228,-0.658075,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.610000,610.0,688.5
2,2,2,2,2,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.088889,560.0,595.0
3,3,3,3,3,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.233333,590.0,635.0
4,4,4,4,4,0.154575,-0.011188,-0.247025,-0.604513,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.383333,482.2,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,210,214,214,214,0.154575,-0.376755,0.713993,4.491519,-0.370637,-0.096002,...,0,0,1,0,0,0,0,4.000000,585.0,630.0
211,211,215,215,215,7.590031,7.504453,-0.439228,5.524499,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.833333,465.0,550.0
212,212,216,216,216,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,1.933333,465.0,595.0
213,213,217,217,217,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,3.033333,495.0,595.0


In [3]:
# Input for ML models

input_col = ['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol',
       'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml',
       'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml',
       'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate',
       'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide',
       'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate',
       'x0_indium trifluoroacetate',
       'x0_indium tris(N,N-diisopropylacetamidinato)',
       'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas',
       'x1_phosphorus trichloride', 'x1_sodium phosphide',
       'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine',
       'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine',
       'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid',
       'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid',
       'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None',
       'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene',
       'x3_octadecene', 'x3_oleylamine', 'x3_trioctylamine',
       'x3_trioctylphosphine', 'x3_trioctylphosphine oxide', 'x4_None',
       'x4_dioctyl ether', 'x4_dioctylamine', 'x4_hexadecylamine',
       'x4_octylamine', 'x4_oleylamine', 'x4_toluene', 'x4_trioctylphosphine',
       'x4_trioctylphosphine oxide', 'x5_None', 'x5_trioctylphosphine',
       'x6_None', 'x6_acetic acid', 'x6_superhydride',
       'x6_tetrabutylammonium myristate', 'x6_zinc acetate', 'x6_zinc bromide',
       'x6_zinc chloride', 'x6_zinc iodide', 'x6_zinc octanoate',
       'x6_zinc oleate', 'x6_zinc stearate', 'x6_zinc undecylenate', 'x7_None',
       'x7_copper bromide', 'x7_oleic acid', 'x7_water', 'x7_zinc iodide']

output_col = ['diameter_nm', 'emission_nm', 'abs_nm']

#Three individual outputs:
diameter = ['diameter_nm']
emission = ['emission_nm']
absorbance = ['abs_nm']

#Splitting dataset

X = df[input_col]

X = df[input_col]

Y = df['abs_nm']

all_col = ['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol',
       'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml',
       'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml',
       'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate',
       'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide',
       'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate',
       'x0_indium trifluoroacetate',
       'x0_indium tris(N,N-diisopropylacetamidinato)',
       'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas',
       'x1_phosphorus trichloride', 'x1_sodium phosphide',
       'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine',
       'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine',
       'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid',
       'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid',
       'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None',
       'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene',
       'x3_octadecene', 'x3_oleylamine', 'x3_trioctylamine',
       'x3_trioctylphosphine', 'x3_trioctylphosphine oxide', 'x4_None',
       'x4_dioctyl ether', 'x4_dioctylamine', 'x4_hexadecylamine',
       'x4_octylamine', 'x4_oleylamine', 'x4_toluene', 'x4_trioctylphosphine',
       'x4_trioctylphosphine oxide', 'x5_None', 'x5_trioctylphosphine',
       'x6_None', 'x6_acetic acid', 'x6_superhydride',
       'x6_tetrabutylammonium myristate', 'x6_zinc acetate', 'x6_zinc bromide',
       'x6_zinc chloride', 'x6_zinc iodide', 'x6_zinc octanoate',
       'x6_zinc oleate', 'x6_zinc stearate', 'x6_zinc undecylenate', 'x7_None',
       'x7_copper bromide', 'x7_oleic acid', 'x7_water', 'x7_zinc iodide',
           'diameter_nm', 'abs_nm', 'emission_nm'            ]

In [4]:
df2 = df.sort_values(by = 'emission_nm')


In [72]:
bins = [450, 530, 560, 580, 620, 800]
df2['binned'] = pd.cut(df2['emission_nm'], bins)
df2

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,...,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm,binned,class
183,183,187,187,187,-0.299814,0.426616,-0.266245,0.084140,1.097051,-0.096002,...,1,0,0,0,0,6.333333,405.0,470.0,"(450, 530]",AA
184,184,188,188,188,-0.299814,0.426616,0.079721,0.084140,1.097051,-0.096002,...,1,0,0,0,0,6.333333,416.0,475.0,"(450, 530]",AA
95,95,95,95,95,-0.506354,-0.594927,-0.170143,-0.225754,-0.273281,-0.096002,...,1,0,0,0,0,3.033333,408.0,475.0,"(450, 530]",AA
168,168,172,172,172,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,...,1,0,0,0,0,2.000000,430.0,477.0,"(450, 530]",AA
167,167,171,171,171,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,...,0,0,1,0,0,2.000000,430.0,477.0,"(450, 530]",AA
161,161,165,165,165,-0.588970,-0.667895,-0.323906,0.015274,-0.370637,-0.096002,...,1,0,0,0,0,1.883333,460.0,480.0,"(450, 530]",AA
154,154,158,158,158,-0.588970,-0.667895,-0.420008,-0.122456,-0.370637,-0.096002,...,1,0,0,0,0,2.300000,436.0,481.0,"(450, 530]",AA
14,14,14,14,14,-0.423738,-0.522690,-0.439228,-0.581375,-0.370637,-0.096002,...,1,0,0,0,0,5.266667,430.0,482.0,"(450, 530]",AA
58,58,58,58,58,-0.423738,-0.682488,-0.439228,-0.329052,-0.370637,-0.096002,...,0,0,0,1,0,1.983333,438.0,483.5,"(450, 530]",AA
61,61,61,61,61,-0.588970,-0.667895,-0.439228,-0.329052,-0.370637,-0.096002,...,1,0,0,0,0,2.300000,423.0,485.0,"(450, 530]",AA


In [73]:
from sklearn.model_selection import StratifiedShuffleSplit

df2 = df.sort_values(by = 'emission_nm')
bins = [450, 530, 560, 580, 620, 800]
df2['binned'] = pd.cut(df2['emission_nm'], bins)
X = df2[input_col]

Y = df2[output_col]

Z = df2[all_col]
Bin = df2["binned"]
print(df2.groupby('binned').size())

binned
(450, 530]    38
(530, 560]    35
(560, 580]    32
(580, 620]    72
(620, 800]    38
dtype: int64


In [7]:
    df2.loc[(df2['emission_nm'] > 450) & (df2['emission_nm'] < 530), 'class'] = "AA"
    df2.loc[(df2['emission_nm'] > 529.999) & (df2['emission_nm'] < 560), 'class'] = "AB"
    df2.loc[(df2['emission_nm'] > 559.99) & (df2['emission_nm'] < 580), 'class'] = "AC"
    df2.loc[(df2['emission_nm'] > 579.99) & (df2['emission_nm'] < 620), 'class'] = "AD"
    df2.loc[(df2['emission_nm'] > 619.99) & (df2['emission_nm'] < 800.001), 'class'] = "AF"

In [8]:
Z.loc[(Z['emission_nm'] > 450) & (Z['emission_nm'] < 530), 'class'] = "AA"
Z.loc[(Z['emission_nm'] > 529.999) & (Z['emission_nm'] < 560), 'class'] = "AB"
Z.loc[(Z['emission_nm'] > 559.99) & (Z['emission_nm'] < 580), 'class'] = "AC"
Z.loc[(Z['emission_nm'] > 579.99) & (Z['emission_nm'] < 620), 'class'] = "AD"
Z.loc[(Z['emission_nm'] > 619.99) & (Z['emission_nm'] < 800.001), 'class'] = "AF"

/tmp/ipykernel_409/1871742393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Z.loc[(Z['emission_nm'] > 450) & (Z['emission_nm'] < 530), 'class'] = "AA"


In [9]:
Z

,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,other_1_amount_mmol,other_2_amount_mmol,total_volume_ml,temp_c,...,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm,class
183,-0.299814,0.426616,-0.266245,0.084140,1.097051,-0.096002,1.019556,-0.146249,0.217625,-1.413230,...,0,1,0,0,0,0,6.333333,405.0,470.0,AA
184,-0.299814,0.426616,0.079721,0.084140,1.097051,-0.096002,1.019556,-0.146249,0.217625,-1.413230,...,0,1,0,0,0,0,6.333333,416.0,475.0,AA
95,-0.506354,-0.594927,-0.170143,-0.225754,-0.273281,-0.096002,-0.213114,-0.146249,-0.244074,-1.811425,...,0,1,0,0,0,0,3.033333,408.0,475.0,AA
168,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,0.249137,-0.146249,0.027912,0.378649,...,0,1,0,0,0,0,2.000000,430.0,477.0,AA
167,-0.175889,-0.448993,0.252705,0.015274,0.118592,-0.096002,0.249137,10.913954,0.027912,0.378649,...,0,0,0,1,0,0,2.000000,430.0,477.0,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,-0.588970,1.448160,-0.323906,-0.535648,-0.370637,-0.096002,-0.675365,-0.146249,-0.541229,-0.417741,...,0,1,0,0,0,0,2.500000,601.0,695.0,AF
27,0.782458,0.017999,-0.439228,-0.508102,-0.261920,-0.096002,-0.675365,-0.146249,-0.501452,1.573235,...,0,1,0,0,0,0,4.600000,650.0,709.0,AF
176,1.253370,1.448160,-0.439228,0.084140,0.118592,-0.096002,-0.675365,-0.146249,0.091149,1.175040,...,0,1,0,0,0,0,4.200000,626.0,729.0,AF
177,1.253370,1.448160,-0.439228,0.084140,0.118592,-0.096002,-0.675365,-0.146249,0.091149,1.374138,...,0,1,0,0,0,0,4.800000,689.0,765.0,AF


In [10]:
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

In [11]:
group = Z['class']
gkf = GroupKFold(n_splits=5)
result = []
for train_index, test_index in gkf.split(Z[input_col], Z[output_col], groups=Z["emission_nm"]):
    train_fold = Z.iloc[train_index]
    test_fold = Z.iloc[test_index]
    result.append((train_fold, test_fold))

train_fold_1, test_fold_1 = result[0][0],result[0][1]
train_fold_2, test_fold_2 = result[1][0],result[1][1]
train_fold_3, test_fold_3 = result[2][0],result[2][1]
train_fold_4, test_fold_4 = result[3][0],result[3][1]
train_fold_5, test_fold_5 = result[4][0],result[4][1]



In [12]:
pd.set_option('display.max_rows', None)
test_fold_1

,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,other_1_amount_mmol,other_2_amount_mmol,total_volume_ml,temp_c,...,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm,class
161,-0.588970,-0.667895,-0.323906,0.015274,-0.370637,-0.096002,-0.575211,-0.146249,-0.035326,1.573235,...,0,1,0,0,0,0,1.883333,460.0,480.0,AA
14,-0.423738,-0.522690,-0.439228,-0.581375,-0.370637,-0.096002,0.457150,-0.146249,-0.583219,-0.815937,...,0,1,0,0,0,0,5.266667,430.0,482.0,AA
180,-0.299814,0.426616,0.079721,0.084140,1.097051,-0.096002,1.019556,-0.146249,0.217625,-1.413230,...,0,1,0,0,0,0,3.033333,448.0,501.0,AA
121,-0.588970,-0.667895,-0.328519,-0.208538,-0.370637,-0.096002,-0.598323,-0.146249,-0.240849,1.573235,...,0,1,0,0,0,0,1.883333,457.0,522.0,AA
197,-0.258506,-0.303058,-0.439228,0.359601,-0.209192,10.377182,-0.290156,-0.146249,0.318679,0.179552,...,1,1,0,0,0,0,1.916667,462.0,522.0,AA
46,0.815505,2.061086,-0.439228,-0.397917,0.265361,-0.096002,2.714477,-0.146249,-0.335769,-0.815937,...,0,1,0,0,0,0,2.200000,444.0,522.5,AA
18,-0.423738,-0.259277,-0.439228,-0.537370,-0.370637,-0.096002,-0.112960,-0.146249,-0.542810,-0.815937,...,0,1,0,0,0,0,3.100000,505.0,530.0,AB
119,-0.588970,-0.667895,-0.343895,-0.208538,-0.370637,-0.096002,-0.598323,-0.146249,-0.240849,1.573235,...,0,1,0,0,0,0,1.883333,470.0,530.0,AB
6,0.154575,-0.011188,-0.247025,-0.604513,-0.370637,-0.096002,0.865472,-0.146249,-0.596878,-0.815937,...,0,1,0,0,0,0,2.383333,482.2,538.0,AB
170,-0.299814,0.426616,-0.439228,0.084140,-0.370637,-0.096002,1.019556,-0.146249,0.027912,-1.413230,...,0,1,0,0,0,0,3.500000,478.0,538.0,AB


In [27]:
train_kf = [train_fold_1, train_fold_2, train_fold_3, train_fold_4, train_fold_5]

test_kf = [ test_fold_1, test_fold_2, test_fold_3, test_fold_4, test_fold_5]
outputs = ('diameter: ', 'absorbance: ', 'emission: ')


for i in range(0, 5):
    X_train = train_kf[i][input_col]
    Y_train = train_kf[i][output_col]
    X_test = test_kf[i][input_col]
    Y_test = test_kf[i][output_col]
    ET_regr = ExtraTreesRegressor(n_estimators=22, max_features=14, random_state=23)
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    for n in range(0,3):
        ET_MAE = np.round(mean_absolute_error(Y_test.iloc[:, n], pd.DataFrame(ET_Y_pred).loc[:, n]), 3)
        print("Fold # ", i+1, outputs[n] ,": ", ' MAE:', ET_MAE )




Fold #  1 diameter:  :   MAE: 0.524
Fold #  1 absorbance:  :   MAE: 21.521
Fold #  1 emission:  :   MAE: 26.989
Fold #  2 diameter:  :   MAE: 0.495
Fold #  2 absorbance:  :   MAE: 28.209
Fold #  2 emission:  :   MAE: 35.342
Fold #  3 diameter:  :   MAE: 0.631
Fold #  3 absorbance:  :   MAE: 31.488
Fold #  3 emission:  :   MAE: 38.933
Fold #  4 diameter:  :   MAE: 0.596
Fold #  4 absorbance:  :   MAE: 25.777
Fold #  4 emission:  :   MAE: 30.675
Fold #  5 diameter:  :   MAE: 0.842
Fold #  5 absorbance:  :   MAE: 29.584
Fold #  5 emission:  :   MAE: 31.716


### SINGLE OUTPUT

In [28]:
train_kf[1]['emission_nm']

183    470.0
168    477.0
167    477.0
161    480.0
154    481.0
14     482.0
61     485.0
103    491.0
96     491.0
97     491.0
99     494.0
98     494.0
4      500.0
180    501.0
179    502.0
172    502.0
123    507.0
54     507.0
57     507.0
17     518.0
120    518.0
29     518.0
11     518.0
121    522.0
197    522.0
46     522.5
201    528.0
12     529.0
18     530.0
119    530.0
49     531.0
67     534.0
68     534.0
6      538.0
170    538.0
0      539.0
7      540.0
188    542.0
81     544.0
80     544.0
149    546.0
145    546.0
15     546.5
133    548.0
187    548.0
100    548.5
47     550.0
156    550.0
211    550.0
151    552.5
53     552.5
134    554.0
185    554.0
117    554.5
62     556.5
186    558.0
160    558.0
10     560.0
25     560.0
82     561.0
150    561.0
83     563.5
158    565.0
69     568.0
37     568.0
144    568.5
137    568.5
114    568.5
132    568.5
136    568.5
135    568.5
43     572.0
195    572.0
140    573.0
126    573.0
125    573.0
138    573.0

## Emission

In [37]:
train_kf = [train_fold_1, train_fold_2, train_fold_3, train_fold_4, train_fold_5]

test_kf = [ test_fold_1, test_fold_2, test_fold_3, test_fold_4, test_fold_5]


for i in range(0, 5):
    X_train = train_kf[i][input_col]
    Y_train = train_kf[i]['emission_nm']
    X_test = test_kf[i][input_col]
    Y_test = test_kf[i]['emission_nm']
    ET_regr = ExtraTreesRegressor(n_estimators=6, max_features=22, random_state=10)
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = np.round(mean_absolute_error(Y_test, ET_Y_pred), 3)
    print("Fold # ", i+1, ": ", ' MAE:', ET_MAE )

Fold #  1 :   MAE: 23.859
Fold #  2 :   MAE: 29.985
Fold #  3 :   MAE: 29.793
Fold #  4 :   MAE: 25.432
Fold #  5 :   MAE: 28.147


## ABS

In [64]:
train_kf = [train_fold_1, train_fold_2, train_fold_3, train_fold_4, train_fold_5]

test_kf = [ test_fold_1, test_fold_2, test_fold_3, test_fold_4, test_fold_5]


for i in range(0, 5):
    X_train = train_kf[i][input_col]
    Y_train = train_kf[i]['abs_nm']
    X_test = test_kf[i][input_col]
    Y_test = test_kf[i]['abs_nm']
    ET_regr = ExtraTreesRegressor(n_estimators=19, max_features=21, random_state=10)
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = np.round(mean_absolute_error(Y_test, ET_Y_pred), 3)
    print("Fold # ", i+1, ": ", ' MAE:', ET_MAE )

Fold #  1 :   MAE: 26.845
Fold #  2 :   MAE: 34.597
Fold #  3 :   MAE: 34.919
Fold #  4 :   MAE: 29.493
Fold #  5 :   MAE: 31.412


### Diameter

In [71]:
train_kf = [train_fold_1, train_fold_2, train_fold_3, train_fold_4, train_fold_5]

test_kf = [ test_fold_1, test_fold_2, test_fold_3, test_fold_4, test_fold_5]


for i in range(0, 5):
    X_train = train_kf[i][input_col]
    Y_train = train_kf[i]['diameter_nm']
    X_test = test_kf[i][input_col]
    Y_test = test_kf[i]['diameter_nm']
    DT_regr = DecisionTreeRegressor(max_depth=4,
                                max_features=27,
                                random_state=28)
    DT_regr.fit(X_train, Y_train)            
    DT_Y_pred = DT_regr.predict(X_test)
    DT_MAE = np.round(mean_absolute_error(Y_test, DT_Y_pred), 3)
    print("Fold # ", i+1, ": ", ' MAE:', DT_MAE )

Fold #  1 :   MAE: 0.576
Fold #  2 :   MAE: 0.549
Fold #  3 :   MAE: 0.493
Fold #  4 :   MAE: 0.743
Fold #  5 :   MAE: 0.895
